In [7]:
import requests as rq
from bs4 import BeautifulSoup as bs
import pandas as pd
import numpy as np


r = None
downloaded = False

def getRequest(url):
    url = url
    r = rq.get(url)
    return r,url
    
def getIndividualCompanyDetailsLinks(r):
    links = []
    if(r):
        soup = bs(r.content, "lxml")
        table = soup.find("table",{"id":"item"})
        rows = table.find_all('tr')
        for i in range(1,len(rows)):
            for a in rows[i].td.find_all('a', href=True):
                links.append(a['href'])
        return links,soup        
    
    
def addToDataFrame(linkss,url):
    df = pd.DataFrame(columns=("Company name", "Products and services", "Address", "City", "Phone", "Fax", "Web", "Desc") )
    for i in range(0,len(linkss)):
        url_individual = url+linkss[i]
        r1 = rq.get(url_individual)
        s = bs(r1.content, 'lxml')
        table = s.find("table", {"id":"item"})
        rows = table.find_all("tr")
        td = rows[1].find_all('td')
        c_name = td[0].text.strip()
        if(not c_name):
            c_name = "None"
        td = rows[3].find_all('td')
        product_service = td[1].text.strip()
        if(not product_service):
            product_service = "None"
        td = rows[4].find_all('td')
        address = td[1].text.strip()
        if(not address):
            address = "None"
        td = rows[5].find_all('td')
        city = td[1].text.strip()
        if(not city):
            city = "None"
        td = rows[6].find_all('td')
        phone = td[1].text.strip()
        if(not phone):
            phone = "None"
        td = rows[7].find_all('td')
        fax = td[1].text.strip()
        if(not fax):
            fax = "None"
        td = rows[10].find_all('td')
        web = td[1].text.strip()
        if(not web):
            web = "None"
        td = rows[9].find_all('td')
        desc = td[1].text.strip()
        if(not desc):
            desc = "None"
        print(i,c_name)
        df = df.append({'Company name': c_name, 'Products and services': product_service, 'Address': address,'City': city, 'Phone': phone, 'Fax': fax, 
              'Web': web,'Desc': desc}, ignore_index=True)
    return df

def getPagers(soup):
    pagers = []
    for a in soup.find_all('a',{"class":"pager"},href=True):
        pagers.append(a['href'])
    pagers.remove(pagers[-1])    
    return pagers    
    
# 1,3,4,5,6,7,10,9        

def getPagerData(pager):
    req1, url1 = getRequest(pager)
    print(url1)
    linkss1, soup = getIndividualCompanyDetailsLinks(req1)
    df2 = addToDataFrame(linkss1,url1)
    return df2
    


    
##..........................##

yellow_book = "http://www.bdyellowbook.com" 
print(".....................Welocme to BD Yellow Book Scrapper.....................")
print("\nConnecting to the server....")
request, url = getRequest("http://www.bdyellowbook.com/")
if(request.status_code==200):
    b_soup = bs(request.content, 'lxml')
    catalog_table = b_soup.find("table", {"id":"catalog"})
    main_categories_link =  []
    main_categories_name = []
    print("\nThe list below is the list of main catergories. Choose your desired list to download the dataset.\n\n")
    for a in catalog_table.find_all("a",href=True):
        link = yellow_book+a['href']
        print(len(main_categories_link),a.string)
        main_categories_name.append(a.string)
        main_categories_link.append(link)

    select = int(input("Choose your desired option: "))
    print("\nGetting ", main_categories_name[select], " dataset..........")
    req, url = getRequest(main_categories_link[select])
    linkss, soup = getIndividualCompanyDetailsLinks(req)
    df = addToDataFrame(linkss,url)
    
    pagers = getPagers(soup)
    print("\nTotal ",len(pagers), "pagers found....")
    for i in range(0,len(pagers)):
        new_df = getPagerData(pagers[i])
        df = df.append(new_df,ignore_index=True)
    print("\nDataset downloaded successfully.....")
    downloaded = True
else:
    print("\nFailed to connect with the server. Please check your internet connection and try again.")

    
if(downloaded==True):
    print("\nPreparing your data to save....")
    df1 = df[(df['Web']=="None") & (df["City"]=="Dhaka")]
    with pd.ExcelWriter(main_categories_name[select]+'.xlsx') as writer:  
        df.to_excel(writer, sheet_name='Total Data')
        df1.to_excel(writer, sheet_name='Without Web and inside dhaka')
    print("\nDataset saved to your local memory successfully!!....")

    

.....................Welocme to BD Yellow Book Scrapper.....................

Connecting to the server....

The list below is the list of main catergories. Choose your desired list to download the dataset.


0 Agricultural Products & Services
1 Art & Culture
2 Automotive
3 Book, Stationery & Supplies
4 Business & Services
5 Chemical & Paper
6 Childrens Goods
7 Community & Organizations
8 Computer & Internet
9 Consultancy & Construction
10 Education & Career
11 Electrical & Electronic
12 Fashion & Jewelry
13 Food & Beverage
14 Garments & Accessories
15 Health & Medicine
16 Household & Handicrafts
17 Jute Goods & Tea
18 Leather & Leather Goods
19 Machinery & Equipment
20 Mills & Industries
21 News & Media
22 Oil & Gas Industries
23 Plastic & Rubbers
24 Real Estate
25 Recreation & Fitness
26 Shopping & Greetings
27 Telecommunication
28 Tour & Travel
Choose your desired option: 0

Getting  Agricultural Products & Services  dataset..........
0 A & A ENTERPRISE
1 AARUSH GROUP
2 ADVANCE AGROT

In [133]:
df1.count()

Company name             29
Products and services    29
Address                  29
City                     29
Phone                    29
Fax                      29
Web                      29
Desc                     29
dtype: int64